In [36]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [37]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [6e4b80f9] BenchmarkTools v1.6.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.19
  [682c06a0] JSON v0.21.4
  [33e6dc65] MKL v0.9.0
→ [da04e1cc] MPI v0.20.23
  [3da0fdf6] MPIPreferences v0.1.11
⌅ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.16
⌅ [a759f4b9] TimerOutputs v0.5.25
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with → are not downloaded, use `instantiate` to download
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [38]:
include("global_adder_passthrough.jl")

filter_highest_tag_number (generic function with 1 method)

In [39]:
# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 14
ancilla = 0
folded = true
seed_vec = nothing
i1 = 1
_maxdim = 2^9
_cutoff = 1e-30
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
rng_vec = MersenneTwister(seed_vec)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);

initial_state = randomMPS(qubit_site; linkdims = _maxdim)
include("run_CT_MPS_1-3.jl")
orthogonalize!(initial_state, div(L,2))


MPS
[1] ((dim=2|id=110|"Qubit,Site,n=1"), (dim=2|id=702|"Link,l=1"))
[2] ((dim=2|id=389|"Qubit,Site,n=2"), (dim=2|id=702|"Link,l=1"), (dim=4|id=542|"Link,l=2"))
[3] ((dim=2|id=398|"Qubit,Site,n=3"), (dim=4|id=542|"Link,l=2"), (dim=8|id=356|"Link,l=3"))
[4] ((dim=2|id=982|"Qubit,Site,n=4"), (dim=8|id=356|"Link,l=3"), (dim=16|id=498|"Link,l=4"))
[5] ((dim=2|id=510|"Qubit,Site,n=5"), (dim=16|id=498|"Link,l=4"), (dim=32|id=455|"Link,l=5"))
[6] ((dim=2|id=319|"Qubit,Site,n=6"), (dim=32|id=455|"Link,l=5"), (dim=64|id=847|"Link,l=6"))
[7] ((dim=2|id=567|"Qubit,Site,n=7"), (dim=128|id=999|"Link,l=7"), (dim=64|id=847|"Link,l=6"))
[8] ((dim=128|id=999|"Link,l=7"), (dim=2|id=210|"Qubit,Site,n=8"), (dim=64|id=329|"Link,l=8"))
[9] ((dim=64|id=329|"Link,l=8"), (dim=2|id=146|"Qubit,Site,n=9"), (dim=32|id=186|"Link,l=9"))
[10] ((dim=32|id=186|"Link,l=9"), (dim=2|id=386|"Qubit,Site,n=10"), (dim=16|id=354|"Link,l=10"))
[11] ((dim=16|id=354|"Link,l=10"), (dim=2|id=768|"Qubit,Site,n=11"), (dim=8|id=30|"Li

In [40]:
# Do this (correct):
site = div(L,2)
left_indices = [siteind(initial_state, site)]  # Just the physical site
if site > 1
    left_link = linkind(initial_state, site-1)
    if !isnothing(left_link)
        push!(left_indices, left_link)
    end
end

println(left_indices)

Index{Int64}[(dim=2|id=567|"Qubit,Site,n=7"), (dim=64|id=847|"Link,l=6")]


In [41]:
tensor = (initial_state[div(L,2)] * initial_state[div(L,2)+1]);
U, S, V = svd(tensor, left_indices);

# fieldnames(typeof(mid_link))
# typeof(sv)
# norm(sv)
# println(norm)
println(array(diag(S)))

[0.17451361581772223, 0.17291823557292724, 0.17164610889442816, 0.16691431611801882, 0.16067781238108528, 0.1583237795021796, 0.15589205351064656, 0.1544654284843477, 0.15228632425551078, 0.15142025821798297, 0.14845989281527805, 0.1470368431929347, 0.14366963310213238, 0.14187347855599006, 0.13999071521458925, 0.1395754417717004, 0.13815671987674205, 0.1357832210949362, 0.13285821137691936, 0.13201731313103104, 0.13040095151532718, 0.12927365450696315, 0.12697802353415272, 0.12490632477746665, 0.12444486794512527, 0.1213462837585368, 0.12096455379497023, 0.11808208002163931, 0.11712856789714321, 0.11618109413199353, 0.11574945999066506, 0.11354550811272346, 0.11180201765027387, 0.11021723259918206, 0.10941889195519458, 0.1078856218337772, 0.105745003172592, 0.10473715896367802, 0.10311688608450312, 0.10123526882781814, 0.09881666499164404, 0.09771185434284103, 0.09612080577866045, 0.09548409696468471, 0.09408885419329197, 0.0929440850068164, 0.09177254959610041, 0.08922053777568961, 0

In [42]:
# manufacture the singular values
rk = minimum([_maxdim, length(array(diag(S)))]);
println(rk)
# println(rank)
req_sv = exp.([-n for n in 1:rk]);
# req_sv = exp.([-n for n in 1:2^9]);
# norm(req_sv);
req_sv = req_sv / norm(req_sv);
println(req_sv)

128
[0.9298734950321937, 0.3420813417125794, 0.12584469282440092, 0.04629567527063242, 0.017031227147214816, 0.006265438325381285, 0.0023049259498354045, 0.0008479348703670045, 0.000311937806260393, 0.00011475550584731902, 4.22161913624579e-5, 1.5530468886807682e-5, 5.713340215209282e-6, 2.1018204055935187e-6, 7.732165162524779e-7, 2.8445045990349104e-7, 1.0464347623025605e-7, 3.8496183557823703e-8, 1.416195449448545e-8, 5.2098919053267035e-9, 1.9166121226952084e-9, 7.050821966395251e-10, 2.593852444796815e-10, 9.542249878730318e-11, 3.5103975529055725e-11, 1.2914030900525007e-11, 4.750806470955878e-12, 1.7477240296489203e-12, 6.42951739349146e-13, 2.365287265719707e-13, 8.701405575228947e-14, 3.201068220421297e-14, 1.1776071880802504e-14, 4.33217474270436e-15, 1.593718023403117e-15, 5.862960958343942e-16, 2.1568628009655536e-16, 7.934654819026798e-17, 2.91899638071187e-17, 1.0738387573177453e-17, 3.9504320195028826e-18, 1.4532827237204927e-18, 5.346328362664064e-19, 1.9668042903758884

In [43]:
initial_state[div(L,2)] = U;
initial_state[div(L,2)+1] = V * diagITensor(req_sv, S.tensor.inds...);

In [44]:
println(initial_state[div(L,2)+1].tensor.inds)

((dim=2|id=210|"Qubit,Site,n=8"), (dim=64|id=329|"Link,l=8"), (dim=128|id=379|"Link,u"))


In [45]:
# check that the middle bond of initial state 1 has the required singular values
mps_ = orthogonalize(copy(initial_state), div(L,2));
# println(linkind(mps_, div(L,2)))
_, S_verify, V = svd(mps_[div(L,2)], (linkind(mps_, div(L,2)),); cutoff=_cutoff);
println(array(diag(S_verify)))
# length(array(diag(S_verify)))
println(req_sv)

[0.9298734950321936, 0.3420813417125794, 0.12584469282440086, 0.04629567527063241, 0.017031227147214823, 0.00626543832538129, 0.0023049259498354054, 0.0008479348703670042, 0.0003119378062603932, 0.00011475550584731913, 4.221619136245792e-5, 1.5530468886807672e-5, 5.713340215209279e-6, 2.10182040559352e-6, 7.732165162524781e-7, 2.844504599034909e-7, 1.0464347623025597e-7, 3.849618355782371e-8, 1.4161954494485473e-8, 5.209891905326708e-9, 1.916612122695213e-9, 7.050821966395255e-10, 2.593852444797053e-10, 9.542249878730331e-11, 3.5103975529192785e-11, 1.2914030900792697e-11, 4.750806476819631e-12, 1.7477240297533066e-12, 6.429517891423685e-13, 2.365287782606417e-13, 8.701408509497159e-14, 3.201074604226381e-14, 1.177688727822404e-14, 4.3362252282346665e-15, 1.5945325142446955e-15]
[0.9298734950321937, 0.3420813417125794, 0.12584469282440092, 0.04629567527063242, 0.017031227147214816, 0.006265438325381285, 0.0023049259498354045, 0.0008479348703670045, 0.000311937806260393, 0.0001147555058

In [46]:
exp(-100)

3.720075976020836e-44

In [49]:
# Never do this:
a = 0.1 + 0.2
b = 0.3
println(a == b)  # false! Due to roundoff

# Do this instead:
println(abs(a - b) < eps(Float64))  # true

false
true


In [ ]:
# using BenchmarkTools
# BenchmarkTools.DEFAULT_PARAMETERS.samples = 100
# BenchmarkTools.DEFAULT_PARAMETERS.evals = 1
# BenchmarkTools.DEFAULT_PARAMETERS.seconds = 300

# @benchmark initial_state_1 = apply(add1_3,$initial_state_1)

# @benchmark initial_state_2 = global_adder($initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)
# BenchmarkTools.DEFAULT_PARAMETERS.seconds
# println(contract(initial_state_2).tensor)
# # The `seconds` expression helps set a rough time budget, see Manual for more explanation
# @btime sin(x) setup=(x=rand()) seconds=3
# # psi = contract(initial_state)
# # println(inds(psi))
# # print_nonzero_coordinates(psi)
# # function perm(bit_string::String, pattern::Vector{Int})
# #     permuted = ['0' for _ in 1:length(bit_string)]
# #     for (i, pos) in enumerate(pattern)
# #         permuted[pos] = bit_string[i]
# #     end
# #     return join(permuted)
# # end